In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import riiideducation
import pandas as pd

# You can only call make_env() once, so don't lose it!
# env = riiideducation.make_env()

# до сюда не трогать

In [ ]:
train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', low_memory=False,  nrows=1000000,
                       dtype={'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 'content_type_id': 'int8',
                              'task_container_id': 'int16', 'user_answer': 'int8', 'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                             'prior_question_had_explanation': 'boolean',
                             }
                      )
train_df

In [ ]:
import math

In [ ]:
# task_container_df = pd.DataFrame()
# task_container_df['task_container_id'] = train_df_q['task_container_id']
# task_container_df['answered_correctly'] = train_df_q['answered_correctly']
# task_container_df = task_container_df.groupby('task_container_id', as_index=False).mean()
# task_container_df

# 1) Working with questions

In [ ]:
# # compute difficulty

# questions['amount_of_right_answers'] = 0
# questions['answered'] = 0

# for i in range(len(train_df)):
#     if train_df.loc[i, 'answered_correctly'] == 1:
#         a = int(train_df.loc[i, 'content_id'])
#         questions.loc[a,'amount_of_right_answers'] += 1
#         questions.loc[a,'answered'] += 1
#     if train_df.loc[i, 'answered_correctly'] == 0:
#         a = int(train_df.loc[i, 'content_id'])
#         questions.loc[a, 'answered'] += 1
# questions['difficulty'] = questions['amount_of_right_answers'] / questions['answered']
# # questions['difficulty'] = questions['difficulty'].fillna(0.5)

In [ ]:
# deal with questions

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
questions = questions.drop(['bundle_id',],axis=1)

questions_ = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
questions_ = questions_.drop(['bundle_id',],axis=1)

amount_of_right_answers = pd.DataFrame()
amount_of_right_answers['question_id'] = train_df['content_id']
amount_of_right_answers['amount_of_right_answers'] = train_df['answered_correctly']
# amount_of_right_answers['prior_question_elapsed_time'] = train_df['prior_question_elapsed_time']
amount_of_right_answers  = amount_of_right_answers[amount_of_right_answers['amount_of_right_answers'] != -1]
amount_of_right_answers = amount_of_right_answers.groupby(['question_id'], as_index=False).sum()

train_df_q = train_df[train_df['content_type_id'] == 0]

# time = pd.DataFrame()
# time['question_id'] = train_df_q['content_id']
# time['prior_question_elapsed_time']  = train_df_q['prior_question_elapsed_time']

total_q = pd.DataFrame()
total_q['answered'] = train_df_q['answered_correctly']
total_q['content_id'] = train_df_q['content_id']
total_q = total_q.groupby('content_id').count()

questions_['answered'] = total_q['answered']
questions_ = questions_.merge(amount_of_right_answers, how='inner', on='question_id')
questions_['difficulty'] = questions_['amount_of_right_answers'] / questions_['answered']
questions_ = questions_.drop(['correct_answer', 'part', 'tags'], axis=1)
questions_df = pd.merge(questions, questions_,how='left', on=['question_id'])
# questions_df = pd.merge(time, questions_,how='left', on=['question_id'])

In [ ]:
questions_df

# Filling missings

In [ ]:
part_stat = questions_df.drop(['question_id'], axis=1).dropna().groupby(['part'], as_index=False).mean()

for row in range (len(questions_df)):
    if math.isnan(questions_df.loc[row, 'difficulty']):
        part = questions_df.loc[row, 'part']
        part_row = part_stat[part_stat['part'] == part]
        questions_df.loc[row,'difficulty'] = float(part_row['difficulty'])
#         questions_df.loc[row,'prior_question_elapsed_time'] = float(part_row['prior_question_elapsed_time'])
        questions_df.loc[row,'amount_of_right_answers'] = int(part_row['amount_of_right_answers'])
        questions_df.loc[row, 'answered'] = int(part_row['answered'])

In [ ]:
questions_df = questions_df.fillna(0)
questions_df

# About tags

In [ ]:
# we only take 1st tag, because it is reasonably connected with tags from lectures
for i in range(len(questions_df)):
    a = str(questions_df.loc[i,'tags']).split(' ')
    questions_df.loc[i, 'tag_1'] = a[0] 

questions_df = questions_df.drop('tags', axis=1)
questions_df

# 2) Lectures prosessing

In [ ]:
# lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
# lectures.groupby(lectures['type_of']).count()
# watched_l = train_df[train_df['content_type_id'] == 1] # df with only lectures

# About time

In [ ]:
answer_time_df = pd.DataFrame()
train_df_q = train_df[train_df['content_type_id'] == 0]
answer_time_df['prior_question_elapsed_time'] = train_df_q['prior_question_elapsed_time']
answer_time_df['answered_correctly'] = train_df['answered_correctly']
answer_time_df = answer_time_df.dropna()
means = answer_time_df.groupby('answered_correctly').mean()
print(means)
proportion_of_right_answers = len(answer_time_df[answer_time_df['answered_correctly'] == 1]) / len(answer_time_df)
print('proportion_of_right_answers in all dataset = ', proportion_of_right_answers)

In [ ]:
tail_size = 1000
answer_time_df_sorted = answer_time_df.sort_values(by=['prior_question_elapsed_time'])
head_stat = answer_time_df_sorted.iloc[:tail_size].groupby('answered_correctly').count() / tail_size
tail_stat = answer_time_df_sorted.iloc[-tail_size:].groupby('answered_correctly').count() / tail_size
print('proportion of right answers in head = ', head_stat['prior_question_elapsed_time'][1])
print('proportion of right answers in tail = ', tail_stat ['prior_question_elapsed_time'][1])

# 3) Creating training dataframe

In [ ]:
# # user_df
# user_df = pd.DataFrame()
# user_df['user_id'] = train_df['user_id'].unique()

# # question_perfomance
# for i in range(len(user_df)):
#     user_q = train_df_q[train_df_q['user_id'] == user_df.loc[i,'user_id']]
#     user_df.loc[i,'answered_all'] = len(user_q)
#     user_df.loc[i,'answered_correctly'] = len(user_q[user_q['answered_correctly'] == 1])
# user_df['perfomance'] = user_df['answered_correctly'] / user_df['answered_all']
# user_df

In [ ]:
user_df = pd.DataFrame()
user_df['user_id'] = train_df_q['user_id']
user_df['performance'] = train_df_q['answered_correctly']
user_df = user_df.groupby('user_id', as_index=False).mean()
user_df

In [ ]:
# # We transform all the features from test batch to answer_df, to actually predict
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


mmscaler = MinMaxScaler()
ohe = OneHotEncoder()

mean_perfomance = user_df['performance'].mean()
mean_difficulty = questions_df['difficulty'].mean()


df = train_df.drop(['timestamp','task_container_id', 'user_answer'], axis=1)
df['question_id'] = df['content_id']
df = df.merge(user_df, on='user_id')
df = df.merge(questions_df[['question_id','difficulty','part']], on='question_id')  

df['performance'] = df['performance'].fillna(value=mean_perfomance)
df['difficulty'] = df['difficulty'].fillna(value=mean_difficulty)     
    
feature_df = pd.DataFrame(df[['answered_correctly','prior_question_elapsed_time', 'performance', 'difficulty', 'part', 'prior_question_had_explanation']])

feature_df['prior_question_elapsed_time'] = df['prior_question_elapsed_time'].dropna()
feature_df['prior_question_elapsed_time'] = mmscaler.fit_transform(feature_df['prior_question_elapsed_time'].to_numpy().reshape(-1, 1))



ohe.fit(feature_df['part'].to_numpy().reshape(-1, 1))
transformed = pd.DataFrame(ohe.transform(feature_df['part'].to_numpy().reshape(-1, 1)).toarray().astype('int'), columns = [f'part {x}' for x in range(1, 8)])
feature_df = pd.concat([feature_df, transformed], axis=1).drop(['part'], axis=1)


    
feature_df['performance'] =  feature_df['performance'].fillna(value=mean_perfomance)
feature_df['difficulty'] =  feature_df['difficulty'].fillna(value=mean_difficulty)   
feature_df['prior_question_elapsed_time'] =  feature_df['prior_question_elapsed_time'].fillna(
        value=feature_df['prior_question_elapsed_time'].mean())

feature_df['prior_question_had_explanation'] = df['prior_question_had_explanation'].fillna(False).astype('int')



feature_df = feature_df.dropna()
feature_df

# 4) Training the model

In [ ]:
answer_df_q = feature_df[feature_df['answered_correctly'] != -1]
y = answer_df_q['answered_correctly']
X = answer_df_q.drop(['answered_correctly'], axis=1)


import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# log_reg_clf = sklearn.linear_model.LogisticRegression()
# log_reg_clf.fit(X_train,y_train)
# log_reg_pred = log_reg_clf.predict(X_test)

# print(cross_val_score(log_reg_clf, X, y, cv=5))
# print(roc_auc_score(y_test, log_reg_pred))
# print(accuracy_score(y_test, log_reg_pred))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf_clf = RandomForestClassifier()
# rf_clf.fit(X_train,y_train)
# rf_pred = rf_clf.predict(X_test)
# print(cross_val_score(rf_clf, X, y, cv=5))
# print(roc_auc_score(y_test, rf_pred))
# print(accuracy_score(y_test, rf_pred))

In [ ]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)
xgb_pred = xgb_clf.predict(X_test)
print(cross_val_score(xgb_clf, X, y, cv=5))
print(roc_auc_score(y_test, xgb_pred))
print(accuracy_score(y_test, xgb_pred))

In [ ]:
# import lightgbm as lgb

In [ ]:
# params={}
# params['learning_rate']=0.03
# params['boosting_type']='gbdt' 
# params['objective']='binary' 
# params['metric']='binary_logloss'
# params['num_leaves']=31
# params['tree_learner']='serial'
# params['max_depth']=10

# lgb_train = lgb.Dataset(X_train, label=y_train)
# lgb_clf=lgb.train(params,lgb_train,80)
# lgb_pred=lgb_clf.predict(X_test)
# lgb_pred=lgb_pred.round(0)
# lgb_pred=lgb_pred.astype(int)
# print(accuracy_score(y_test, lgb_pred))

In [ ]:
def batch_processing(test_df, model, alpha):
    df = test_df.drop(['timestamp','task_container_id'], axis=1)
    df['question_id'] = df['content_id']
    df = df.merge(user_df, on='user_id', how='left')
    df = df.merge(questions_df[['question_id','difficulty','part']], on='question_id', how='left')  

    df['performance'] = df['performance'].fillna(value=mean_perfomance)
    df['difficulty'] = df['difficulty'].fillna(value=mean_difficulty)     

    feature_df = pd.DataFrame(df[['prior_question_elapsed_time', 'performance', 'difficulty', 'part', 'prior_question_had_explanation']])


    feature_df['prior_question_elapsed_time'] = df['prior_question_elapsed_time'].dropna()
    feature_df['prior_question_elapsed_time'] = mmscaler.fit_transform(feature_df['prior_question_elapsed_time'].to_numpy().reshape(-1, 1))

    ohe.fit(feature_df['part'].to_numpy().reshape(-1, 1))
    transformed = ohe.transform(feature_df['part'].to_numpy().reshape(-1, 1)).toarray().astype('int') 
    enc = pd.DataFrame(0, index=np.arange(len(feature_df)), columns=[f'part {x}' for x in range(1,8)])
    for i in range(len(transformed.T)):
        enc[f'part {i+1}'] = transformed.T[i]
    
    
    feature_df = pd.concat([feature_df, enc], axis=1).drop(['part'], axis=1)



    feature_df['performance'] =  feature_df['performance'].fillna(value=mean_perfomance)
    feature_df['difficulty'] =  feature_df['difficulty'].fillna(value=mean_difficulty)   


    feature_df['prior_question_elapsed_time'] =  feature_df['prior_question_elapsed_time'].fillna(
            value=feature_df['prior_question_elapsed_time'].mean())

    feature_df['prior_question_had_explanation'] = df['prior_question_had_explanation'].fillna(False).astype('int')
    # feature_df = feature_df.drop(['content_id', 'answered_correctly'], axis=1)

    print(f'With model: {str(model)} and alpha = {alpha}')
    A = model.predict(feature_df)
    submit = pd.DataFrame(test_df['row_id'])

    submit['answered_correctly'] = A
    submit['answered_correctly'] = submit['answered_correctly'].apply(lambda x: 1 if x >= alpha else 0)
    submit['group_num'] = test_df.index

    print('All_submitted')
    return submit

In [ ]:
env = riiideducation.make_env()

iter_test = env.iter_test()

In [ ]:
# models = [log_reg_clf, rf_clf, xgb_clf, lgb_clf]

for (test_df, sample_prediction_df) in iter_test:
    env.predict(batch_processing(test_df,  xgb_clf, 0.75))

In [ ]:
batch_processing(test_df,  xgb_clf, 0.75)

In [ ]:
test_df.index